In [7]:
import os
import numpy as np
from scipy.io import loadmat

folder_path = r"..\los_data"  # 你的文件夹路径
file_list = [f for f in os.listdir(folder_path) if f.endswith('.mat')]  # 找到所有mat文件
file_list = sorted(file_list)  # 按文件名排序，比如 dev0.mat, dev1.mat, dev2.mat

data = []  # 用来存储每个mat文件中的 'data_Ineed'（转置后的）

for i, file_name in enumerate(file_list):
    full_path = os.path.join(folder_path, file_name)
    mat_contents = loadmat(full_path)
    if 'data_Ineed' in mat_contents:
        data_ineed = mat_contents['data_Ineed'].T  # 加上 .T 转置！
        
        # 数据格式转化，按 320 信号一块进行处理
        num_signals, num_samples = data_ineed.shape
        
        # 确保每 320 个信号作为一个大块进行处理
        block_size = 320
        num_blocks = num_signals // block_size  # 计算可以完整分成多少块

        # 舍去多余的信号
        data_ineed = data_ineed[:num_blocks * block_size, :]
        
        # 将信号数据重构：每 320 个信号为一个大块
        reshaped_data = data_ineed.reshape((block_size, num_blocks * num_samples)).T  # 转置回原来的格式

        data.append(reshaped_data)
    else:
        print(f"Warning: {file_name} 里没有 'data_Ineed' 变量！")

# 打印检查
for i, d in enumerate(data):
    print(f"第{i}个文件（{file_list[i]}）的数据 shape (处理后): {d.shape}")


第0个文件（dev0.mat）的数据 shape (处理后): (6720, 320)
第1个文件（dev1.mat）的数据 shape (处理后): (6400, 320)
第2个文件（dev10.mat）的数据 shape (处理后): (8640, 320)
第3个文件（dev11.mat）的数据 shape (处理后): (7680, 320)
第4个文件（dev13.mat）的数据 shape (处理后): (7360, 320)
第5个文件（dev14.mat）的数据 shape (处理后): (6080, 320)
第6个文件（dev15.mat）的数据 shape (处理后): (5440, 320)
第7个文件（dev16.mat）的数据 shape (处理后): (12800, 320)
第8个文件（dev17.mat）的数据 shape (处理后): (9280, 320)
第9个文件（dev18.mat）的数据 shape (处理后): (3200, 320)
第10个文件（dev19.mat）的数据 shape (处理后): (4480, 320)
第11个文件（dev2.mat）的数据 shape (处理后): (6080, 320)
第12个文件（dev20.mat）的数据 shape (处理后): (5440, 320)
第13个文件（dev3.mat）的数据 shape (处理后): (5440, 320)
第14个文件（dev4.mat）的数据 shape (处理后): (6400, 320)
第15个文件（dev5.mat）的数据 shape (处理后): (5120, 320)
第16个文件（dev6.mat）的数据 shape (处理后): (5440, 320)
第17个文件（dev7.mat）的数据 shape (处理后): (8640, 320)
第18个文件（dev8.mat）的数据 shape (处理后): (7360, 320)
第19个文件（dev9.mat）的数据 shape (处理后): (7040, 320)


In [8]:
import numpy as np

# 假设data是从之前的加载代码中获取的
# data = <从前一部分代码中加载的数据>

SNR_dB = 0  # 设定信噪比（dB）

noisy_data = []  # 用来存储加噪声后的数据

for d in data:
    # 计算信号的标准差
    signal_std = np.std(d)
    
    # 计算噪声的标准差，根据信噪比（SNR）
    noise_std = signal_std / (10 ** (SNR_dB / 20))  # 根据 SNR(dB) 计算噪声标准差
    
    # 生成高斯噪声并加到原信号上
    noise = np.random.normal(0, noise_std, d.shape)  # 均值为0，标准差为噪声标准差
    noisy_signal = d + noise  # 添加噪声
    
    noisy_data.append(noisy_signal)  # 保存加噪声后的信号

# 打印检查加噪声后的数据
print("原始数据和加噪声数据示例：")
print(f"原始信号样本（第一个信号）：{data[0][0][:10]}")  # 打印前10个采样点
print(f"加噪声信号样本（第一个信号）：{noisy_data[0][0][:10]}")  # 打印前10个采样点


原始数据和加噪声数据示例：
原始信号样本（第一个信号）：[-0.01318091-0.00598181j  0.00031621-0.0016488j  -0.00036697+0.00361327j
  0.00384329-0.00022767j -0.00345926+0.00189173j  0.00564349-0.00026191j
 -0.00387471-0.00283339j  0.00168507-0.00295063j  0.00239012-0.00275673j
 -0.00055844-0.00026162j]
加噪声信号样本（第一个信号）：[ 1.27847454-0.00598181j -0.54489141-0.0016488j   1.22673463+0.00361327j
 -1.60750186-0.00022767j -0.28652356+0.00189173j -0.77900424-0.00026191j
  0.65713987-0.00283339j  0.65450788-0.00295063j  1.97877183-0.00275673j
  0.84880351-0.00026162j]


In [ ]:
import numpy as np

def compute_doppler_shift(v, fc):
    """
    根据移动速度 v (m/s) 和载波频率 fc (Hz) 计算多普勒频移
    """
    c = 3e8  # 光速 m/s
    return (v / c) * fc

def add_doppler_shift(signal, fd, fs):
    """
    给IQ信号加多普勒频移
    signal: shape (num_channels, num_samples)，复数IQ信号
    fd: 多普勒频移 (Hz)
    fs: 采样率 (Hz)
    """
    num_channels, num_samples = signal.shape
    t = np.arange(num_samples) / fs  # 时间轴
    doppler_phase = np.exp(1j * 2 * np.pi * fd * t)  # 复指数
    return signal * doppler_phase

# 例子
fs = 20e6  # 采样率，比如1 MHz
fc = 2.4e9  # 载波频率，比如2.4 GHz Wi-Fi
v = 120  # 移动速度，比如30 m/s (~108 km/h)

fd = compute_doppler_shift(v, fc)  # 先计算多普勒频移
print(f"计算得到的多普勒频移 fd = {fd:.2f} Hz")

# 假设 data 是你之前读出来的list，每个是 shape (信号数量, 采样点数)
data_with_doppler = []
for sig in noisy_data:
    shifted_sig = add_doppler_shift(sig, fd, fs)
    data_with_doppler.append(shifted_sig)

print("添加多普勒频移后的信号：")
print(data_with_doppler)


In [9]:
import torch
import numpy as np
from torch.utils.data import DataLoader, TensorDataset, random_split

# 假设 data 已经是你的读取好的列表，每个元素是 (总信号数量, 320)

all_data = []
all_labels = []

for device_idx, device_signals in enumerate(noisy_data):
    # device_signals: (总信号数量, 320)，比如 (7014, 320)
    device_labels = np.full((device_signals.shape[0],), device_idx)  # 每条信号的标签都是设备编号
    
    all_data.append(torch.tensor(device_signals, dtype=torch.float32))
    all_labels.append(torch.tensor(device_labels, dtype=torch.long))

# 把20个设备的数据和标签拼接在一起
all_data = torch.cat(all_data, dim=0)  # (总样本数, 320)
all_labels = torch.cat(all_labels, dim=0)  # (总样本数, )

print(f"最终数据 shape: {all_data.shape}")
print(f"最终标签 shape: {all_labels.shape}")

# 构建 PyTorch Dataset
dataset = TensorDataset(all_data, all_labels)

# 划分训练集和验证集，假设验证集占20%
train_size = int(0.8 * len(dataset))  # 80% 训练集
val_size = len(dataset) - train_size  # 剩下的 20% 验证集

# 使用 random_split 划分数据集
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# 打印一下数据集的基本信息
print(f"训练集大小: {len(train_dataset)}")
print(f"验证集大小: {len(val_dataset)}")


最终数据 shape: torch.Size([135040, 320])
最终标签 shape: torch.Size([135040])
训练集大小: 108032
验证集大小: 27008


In [11]:
# ==== 数据增强：对训练集添加高斯噪声 ====

def add_noise_to_dataset(dataset, noise_std=0.01):
    noisy_data = []
    noisy_labels = []
    for x, y in dataset:
        noise = torch.randn_like(x) * noise_std
        noisy_x = x + noise
        noisy_data.append(noisy_x)
        noisy_labels.append(y)
    noisy_data = torch.stack(noisy_data)
    noisy_labels = torch.tensor(noisy_labels)
    return TensorDataset(noisy_data, noisy_labels)

train_dataset = add_noise_to_dataset(train_dataset, noise_std=0.01)  # 可调节 std

In [12]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix
import seaborn as sns
from torch.utils.data import DataLoader, TensorDataset, Subset
from datetime import datetime
from tqdm import tqdm  # 导入tqdm库来显示进度条
from sklearn.model_selection import KFold

# 假设 SNR_dB 和 fd 已经在之前定义
SNR_dB = globals().get('SNR_dB', 'no')
fd = globals().get('fd', 'no')

# 初始化模型参数
input_dim = 320  # 每个信号有320个采样点
num_heads = 4  # 注意力头数
num_layers = 2  # Transformer编码器层数
num_classes = 20  # 有20个设备
dropout = 0.4  # Dropout率

# 训练参数
batch_size = 128
num_epochs = 100  # 训练轮数
learning_rate = 1e-4
patience = 5  # Early stopping 的容忍期（最多允许多少个epoch验证集性能没有改善）

# 训练过程的时间戳和文件夹名
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
script_name = "cross"

# 构造文件夹名
folder_name = f"{timestamp}_{script_name}_SNR{SNR_dB}dB_fd{fd}_classes_{num_classes}_Transformer"
save_folder = os.path.join(os.getcwd(), "training_results", folder_name)

# 创建保存结果的文件夹
os.makedirs(save_folder, exist_ok=True)

# 打开结果文件，保存训练的总结
results_file = os.path.join(save_folder, "results.txt")
with open(results_file, "w") as f:
    f.write(f"=== Experiment Summary ===\n")
    f.write(f"Feature Folder: trajectory_plots\n")
    f.write(f"Timestamp: {timestamp}\n")
    f.write(f"Total Classes: {num_classes}\n")
    f.write(f"SNR: {SNR_dB} dB\n")
    f.write(f"fd (Doppler shift): {fd} Hz\n")


# 定义SignalTransformer模型
class SignalTransformer(nn.Module):
    def __init__(self, input_dim, num_heads, num_layers, num_classes, dropout=0.1):
        super(SignalTransformer, self).__init__()
        
        # 输入数据的嵌入层
        self.embedding = nn.Linear(input_dim, input_dim)  # 将输入信号的维度转换为适合Transformer的维度
        
        # Transformer编码器层
        self.transformer = nn.Transformer(
            d_model=input_dim,  # 输入特征维度
            nhead=num_heads,  # 多头注意力机制的头数
            num_encoder_layers=num_layers,  # 编码器层数
            dropout=dropout,  # Dropout率
            batch_first=True  # 设置为True以支持(batch_size, seq_len, input_dim)输入
        )
        
        # 最后的分类层，将Transformer的输出映射到类别
        self.fc = nn.Linear(input_dim, num_classes)  # 分类输出层
    
    def forward(self, x):
        # Transformer输入要求的格式是 (batch_size, seq_len, input_dim)
        x = self.embedding(x)  # 变为 (batch_size, seq_len, input_dim)
        x = self.transformer(x, x)  # (batch_size, seq_len, input_dim)
        
        # 确认x的维度
        if len(x.shape) == 3:
            x = x[:, -1, :]  # 取序列最后一个时间步的输出 (batch_size, input_dim)
        elif len(x.shape) == 2:
            # 如果是二维的，则直接使用输出 (batch_size, input_dim)
            pass
            
        x = self.fc(x)  # (batch_size, num_classes)
        return x


# K折交叉验证相关参数
n_splits = 5  # 设置K折交叉验证的折数

# 初始化K折交叉验证
kfold = KFold(n_splits=n_splits, shuffle=True, random_state=42)

# 用来记录每一折的训练结果
fold_results = []

# K折交叉验证循环
for fold, (train_idx, val_idx) in enumerate(kfold.split(train_dataset)):
    print(f"\n====== Fold {fold+1}/{n_splits} ======")

    # 打印调试信息：检查数据切分
    print(f"Training indices: {train_idx[:10]}")  # 打印前10个训练样本的索引
    print(f"Validation indices: {val_idx[:10]}")  # 打印前10个验证样本的索引

    # 切分训练集和验证集
    train_subset = Subset(train_dataset, train_idx)
    val_subset = Subset(train_dataset, val_idx)

    # 创建训练和验证加载器
    train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True, drop_last=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, drop_last=True)

    # 初始化模型
    model = SignalTransformer(input_dim=input_dim, num_heads=num_heads, num_layers=num_layers, num_classes=num_classes, dropout=dropout)

    # 损失函数和优化器
    criterion = nn.CrossEntropyLoss()  # 交叉熵损失
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)

    # 学习率调度器
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

    # 用来记录训练过程的损失和准确度
    train_losses = []
    val_losses = []
    train_accuracies = []
    val_accuracies = []

    # Early stopping 变量
    best_val_loss = float('inf')
    patience_counter = 0

    # 训练过程
    for epoch in range(num_epochs):
        model.train()
        running_train_loss = 0.0
        correct_train = 0
        total_train = 0
        
        with tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", unit="batch") as tepoch:
            for batch_idx, (inputs, labels) in enumerate(tepoch):
                optimizer.zero_grad()
                
                # 前向传播
                outputs = model(inputs)
                
                # 计算损失
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                
                running_train_loss += loss.item()
                
                # 计算训练准确度
                _, predicted = torch.max(outputs, 1)
                total_train += labels.size(0)
                correct_train += (predicted == labels).sum().item()

                train_accuracy = 100 * correct_train / total_train
                tepoch.set_postfix(train_loss=running_train_loss / (batch_idx + 1), train_accuracy=train_accuracy)

        epoch_train_loss = running_train_loss / len(train_loader)
        epoch_train_acc = 100 * correct_train / total_train
        train_losses.append(epoch_train_loss)
        train_accuracies.append(epoch_train_acc)

        # 验证过程
        model.eval()
        running_val_loss = 0.0
        correct_val = 0
        total_val = 0
        
        with torch.no_grad():
            with tqdm(val_loader, desc="Validation", unit="batch") as vepoch:
                for val_inputs, val_labels in vepoch:
                    val_outputs = model(val_inputs)
                    val_loss = criterion(val_outputs, val_labels)
                    running_val_loss += val_loss.item()
                    _, val_predicted = torch.max(val_outputs, 1)
                    total_val += val_labels.size(0)
                    correct_val += (val_predicted == val_labels).sum().item()

                    val_accuracy = 100 * correct_val / total_val
                    vepoch.set_postfix(val_loss=running_val_loss / (len(val_loader) + 1), val_accuracy=val_accuracy)

        epoch_val_loss = running_val_loss / len(val_loader)
        epoch_val_acc = 100 * correct_val / total_val
        val_losses.append(epoch_val_loss)
        val_accuracies.append(epoch_val_acc)

        print(f"Epoch {epoch+1}/{num_epochs}:")
        print(f"  Train Loss: {epoch_train_loss:.4f}, Train Accuracy: {epoch_train_acc:.2f}%")
        print(f"  Validation Loss: {epoch_val_loss:.4f}, Validation Accuracy: {epoch_val_acc:.2f}%")

        with open(results_file, "a") as f:
            f.write(f"Epoch {epoch+1} | Train Loss: {epoch_train_loss:.4f} | Train Accuracy: {epoch_train_acc:.2f}% | Validation Loss: {epoch_val_loss:.4f} | Validation Accuracy: {epoch_val_acc:.2f}%\n")

        # Early Stopping 检查
        if epoch_val_loss < best_val_loss:
            best_val_loss = epoch_val_loss
            patience_counter = 0  # 重置计数器
        else:
            patience_counter += 1  # 验证损失没有改善，增加计数器

        if patience_counter >= patience:
            print(f"Early stopping at epoch {epoch+1} due to no improvement in validation loss.")
            break

        # 学习率调度器步进
        scheduler.step()

    # 保存每一折的训练和验证结果
    fold_results.append(max(val_accuracies))

    # 绘制训练和验证损失曲线
    plt.figure()
    plt.plot(range(1, len(train_losses) + 1), train_losses, label='Train Loss')
    plt.plot(range(1, len(val_losses) + 1), val_losses, label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title(f'Fold {fold+1} Training and Validation Loss Curve')
    plt.legend()
    plt.savefig(os.path.join(save_folder, f"fold_{fold+1}_loss_curve.png"))
    plt.close()

    # 绘制训练和验证准确度曲线
    plt.figure()
    plt.plot(range(1, len(train_accuracies) + 1), train_accuracies, label='Train Accuracy')
    plt.plot(range(1, len(val_accuracies) + 1), val_accuracies, label='Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title(f'Fold {fold+1} Training and Validation Accuracy Curve')
    plt.legend()
    plt.savefig(os.path.join(save_folder, f"fold_{fold+1}_accuracy_curve.png"))
    plt.close()

    # 生成并保存混淆矩阵
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for val_inputs, val_labels in val_loader:
            val_outputs = model(val_inputs)
            _, val_predicted = torch.max(val_outputs, 1)
            all_preds.extend(val_predicted.cpu().numpy())
            all_labels.extend(val_labels.cpu().numpy())

    cm = confusion_matrix(all_labels, all_preds)

    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=True)
    plt.title(f'Fold {fold+1} Confusion Matrix')
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.tight_layout()

    confusion_matrix_path = os.path.join(save_folder, f"fold_{fold+1}_confusion_matrix.png")
    plt.savefig(confusion_matrix_path)
    plt.close()

    # 保存混淆矩阵原始数据（保存为 txt 文件）
    np.savetxt(os.path.join(save_folder, f"fold_{fold+1}_confusion_matrix.txt"), cm, fmt='%d')

# 汇总每一折的最佳验证准确率
avg_acc = np.mean(fold_results)
with open(results_file, "a") as f:
    f.write("\n=== Cross-Validation Results ===\n")
    for i, acc in enumerate(fold_results):
        f.write(f"Fold {i+1}: Best Val Accuracy: {acc:.2f}%\n")
    f.write(f"\nAverage Best Val Accuracy: {avg_acc:.2f}%\n")

print(f"\n=== Cross-Validation Summary ===")
for i, acc in enumerate(fold_results):
    print(f"Fold {i+1}: Best Val Accuracy = {acc:.2f}%")
print(f"Average Best Val Accuracy: {avg_acc:.2f}%")



====== Fold 1/5 ======
Training indices: [ 0  1  2  4  5  6  7  8  9 10]
Validation indices: [ 3 23 24 35 39 53 70 87 88 90]


Validation: 100%|██████████| 211/211 [00:09<00:00, 23.08batch/s, val_accuracy=9.43, val_loss=2.95]


Epoch 1/100:
  Train Loss: 3.0059, Train Accuracy: 7.68%
  Validation Loss: 2.9642, Validation Accuracy: 9.43%


Validation: 100%|██████████| 211/211 [00:09<00:00, 22.26batch/s, val_accuracy=9.43, val_loss=2.95]


Epoch 2/100:
  Train Loss: 2.9868, Train Accuracy: 8.21%
  Validation Loss: 2.9600, Validation Accuracy: 9.43%


...

Epoch 14/100:
  Train Loss: 0.0663, Train Accuracy: 97.99%
  Validation Loss: 0.6631, Validation Accuracy: 84.32%


Validation: 100%|██████████| 211/211 [00:11<00:00, 17.79batch/s, val_accuracy=84.6, val_loss=0.658]


Epoch 15/100:
  Train Loss: 0.0582, Train Accuracy: 98.23%
  Validation Loss: 0.6616, Validation Accuracy: 84.59%


Validation: 100%|██████████| 211/211 [00:14<00:00, 14.92batch/s, val_accuracy=84.8, val_loss=0.668]


Epoch 16/100:
  Train Loss: 0.0532, Train Accuracy: 98.34%
  Validation Loss: 0.6712, Validation Accuracy: 84.80%
Early stopping at epoch 16 due to no improvement in validation loss.

=== Cross-Validation Summary ===
Fold 1: Best Val Accuracy = 84.75%
Fold 2: Best Val Accuracy = 84.59%
Fold 3: Best Val Accuracy = 84.68%
Fold 4: Best Val Accuracy = 84.38%
Fold 5: Best Val Accuracy = 84.80%
Average Best Val Accuracy: 84.64%
